In [1]:
from vnpy.trader.object import TickData, TradeData
from vnpy.trader.constant import Direction, Offset
from vnpy.trader.engine import BaseEngine
from vnpy.trader.utility import BarGenerator, ArrayManager
from vnpy.app.cta_strategy import (
    CtaTemplate,
    StopOrder
)
from vnpy.trader.object import OrderData, TradeData, PositionData
from vnpy.trader.event import EVENT_TIMER

class VirtualStopLossStrategy(CtaTemplate):
    author = "WeTab AI"

    # 策略参数
    symbol = ""
    exchange = ""
    stop_loss_points = 20  # 虚拟交易的止损点
    real_stop_loss_points = 15  # 真实交易的止损点
    dynamic_stop_loss_points = 10  # 动态止损点

    # 策略变量
    virtual_position_long = 0  # 虚拟多头持仓
    virtual_position_short = 0  # 虚空头持仓
    real_position_long = 0  # 真实多头持仓
    real_position_short = 0  # 真实空头持仓
    virtual_buy_price = 0  # 虚拟买入价格
    virtual_sell_price = 0  # 虚拟卖出价格
    real_buy_price = 0  # 真实买入价格
    real_sell_price = 0  # 真实卖出价格
    is_virtual_trading = False  # 是否正在进行虚拟交易

    parameters = [
        "symbol", "exchange", "stop_loss_points", "real_stop_loss_points", "dynamic_stop_loss_points"
    ]
    variables = [
        "virtual_position_long", "virtual_position_short", "real_position_long", "real_position_short",
        "virtual_buy_price", "virtual_sell_price", "real_buy_price", "real_sell_price", "is_virtual_trading"
    ]

    def __init__(self, cta_engine: BaseEngine, strategy_name: str, vt_symbol: str, setting: dict):
        super().__init__(cta_engine, strategy_name, vt_symbol, setting)
        self.bg = BarGenerator(self.on_bar)
        self.am = ArrayManager()

    def on_init(self):
        """初始化策略"""
        self.write_log("策略初始化")
        self.load_tick(10)

    def on_start(self):
        """启动策略"""
        self.write_log("策略启动")
        self.put_event()

    def on_stop(self):
        """停止策略"""
        self.write_log("策略停止")
        self.put_event()

    def on_tick(self, tick: TickData):
        """收到行情TICK推送"""
        self.bg.update_tick(tick)

    def on_bar(self, bar):
        """收到Bar推送"""
        self.cancel_all()  # 撤销所有挂单

        # 检查当前是否有持仓
        if self.real_position_long == 0 and self.real_position_short == 0:
            # 没有持仓，进行虚拟交易
            if not self.is_virtual_trading:
                self.virtual_buy_price = bar.close_price
                self.virtual_sell_price = bar.close_price
                self.virtual_position_long = 1
                self.virtual_position_short = 1
                self.is_virtual_trading = True
                self.write_log(f"进行虚拟交易，买入和卖出价格: {self.virtual_buy_price}")

        # 检查虚拟交易是否达到止损点
        if self.is_virtual_trading:
            if bar.close_price - self.virtual_buy_price >= self.stop_loss_points:
                # 虚拟空头发生止损，进行真实多头交易
                self.buy(bar.close_price + 1, 1)
                self.real_position_long = 1
                self.real_buy_price = bar.close_price + 1
                self.is_virtual_trading = False
                self.write_log(f"虚拟空头发生止损，真实买入价格: {self.real_buy_price}")

            elif self.virtual_sell_price - bar.close_price >= self.stop_loss_points:
                # 虚拟多头发生止损，进行真实空头交易
                self.sell(bar.close_price - 1, 1)
                self.real_position_short = 1
                self.real_sell_price = bar.close_price - 1
                self.is_virtual_trading = False
                self.write_log(f"虚拟多头发生止损，真实卖出价格: {self.real_sell_price}")

        # 检查真实交易是否达到止损点
        if self.real_position_long > 0:
            if self.real_buy_price - bar.close_price >= self.real_stop_loss_points:
                # 真实多头发生止损，平仓
                self.sell(bar.close_price - 1, 1)
                self.real_position_long = 0
                self.write_log(f"真实多头发生止损，平仓价格: {bar.close_price - 1}")

        elif self.real_position_short > 0:
            if bar.close_price - self.real_sell_price >= self.real_stop_loss_points:
                # 真实空头发生止损，平仓
                self.buy(bar.close_price + 1, 1)
                self.real_position_short = 0
                self.write_log(f"真实空头发生止损，平仓价格: {bar.close_price + 1}")

        # 动态设置止损点
        if self.real_position_long > 0:
            self.real_stop_loss_points = max(self.real_stop_loss_points, self.dynamic_stop_loss_points)
            if bar.close_price - self.real_buy_price > self.dynamic_stop_loss_points:
                self.real_stop_loss_points = bar.close_price - self.real_buy_price - 1
                self.write_log(f"动态调整真实多头止损点: {self.real_stop_loss_points}")

        elif self.real_position_short > 0:
            self.real_stop_loss_points = max(self.real_stop_loss_points, self.dynamic_stop_loss_points)
            if self.real_sell_price - bar.close_price > self.dynamic_stop_loss_points:
                self.real_stop_loss_points = self.real_sell_price - bar.close_price - 1
                self.write_log(f"动态调整真实空头止损点: {self.real_stop_loss_points}")

        self.put_event()

    def on_order(self, order: OrderData):
        """收到委托变化推送"""
        self.write_log(f"收到委托变化: {order}")
        self.put_event()

    def on_trade(self, trade: TradeData):
        """收到成交推送"""
        self.write_log(f"收到成交: {trade}")
        if trade.direction == Direction.LONG:
            self.real_position_long += trade.volume
            if self.real_position_long > 0:
                self.real_buy_price = trade.price
        elif trade.direction == Direction.SHORT:
            self.real_position_short += trade.volume
            if self.real_position_short > 0:
                self.real_sell_price = trade.price
        self.put_event()

    def on_position(self, position: PositionData):
        """收到持仓推送"""
        self.write_log(f"收到持仓: {position}")
        self.put_event()

    def on_timer(self):
        """定时器事件"""
        self.write_log("定时器触发")
        self.put_event()

    def on_log(self, log: dict):
        """收到日志推送"""
        self.write_log(f"{log['log_time']}: {log['log_msg']}")

# 示例配置
strategy_setting = {
    "symbol": "MA8888",
    "exchange": "CFFEX",
    "stop_loss_points": 20,
    "real_stop_loss_points": 15,
    "dynamic_stop_loss_points": 10
}

# 在 `vn.py` 的 CTA引擎中添加策略
# cta_engine.add_strategy(VirtualStopLossStrategy, "策略名称", "vt_symbol", strategy_setting)

ModuleNotFoundError: No module named 'vnpy.app'